# 14. REINFORCE and Tree Search on Connect 4


## 지금까지 해온 것들을 모두 모으자!

여기서는 policy gradient (REINFORCE),
tree search with policy를 혼용해서
`agent_greedy`를 뛰어넘는
Connect 4 policy를 강화학습 시켜봅니다.

방식은 다음과 같습니다.

- Episode에 대해서는 Monte Carlo policy gradient를
사용하여 policy를 갱신.
- Policy는 policy network를 base policy로 하여
tree search with policy를 사용.

# WIP